<a href="https://colab.research.google.com/github/alifele/EEG-computation/blob/master/colab_1D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install mat4py

In [2]:
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mat4py as mt
import os
from IPython.display import Image
from IPython.core.display import HTML 
from itertools import combinations
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv1D, Dropout, MaxPooling1D
from keras.utils import to_categorical
from sklearn.utils import shuffle

Using TensorFlow backend.


In [3]:
!git clone https://github.com/alifele/EEG-computation.git
%cd EEG-computation

fatal: destination path 'EEG-computation' already exists and is not an empty directory.
/content/EEG-computation


In [4]:
%ls

 ali*  'EEG Data'/   MainCode.ipynb*   Pictures/   README.md*   Untitled.ipynb*


In [0]:
def data_gen():  
    disorder_dic = []
    directory = os.fsencode('EEG Data/disorder')
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        disorder_dic.append(np.array(mt.loadmat('EEG Data/disorder/{}'.format(filename))[filename.split('.')[0]]))

    channels_disorder = [ [] for i in range(len(disorder_dic))]  #channels[person][channel] it will be a numpy array
    j=0;
    for person in channels_disorder:
        for i in range(19):
            person.append(disorder_dic[j][:,i])


    normal_dic = []
    directory = os.fsencode('EEG Data/normal')
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        normal_dic.append(np.array(mt.loadmat('EEG Data/normal/{}'.format(filename))[filename.split('.')[0]]))

    channels_normal = [ [] for i in range(len(normal_dic))]  #channels[person][channel] it will be a numpy array
    j=0;
    for person in channels_normal:
        for i in range(19):
            person.append(normal_dic[j][:,i])

    #normal = channels_normal
    #disorder = channels_disorder

    return channels_normal, channels_disorder

In [0]:
def New_Data_Generator_N(normal):
    number_of_samples = 45
    randlist = (132 * np.random.random(number_of_samples)).astype(int)
    new_patients = list(normal[i] for i in randlist)
    
    index = [ i for i in range(number_of_samples)]
    comb = list( combinations(index, 2))
    new_data = [[] for i in range(len(comb))]
    person = 0
    for x,y in comb:
        for chan in range(19):
            new_data[person].append((new_patients[x][chan] + new_patients[y][chan] )/2)
        normal.append(new_data[person])
        person += 1
    
    return np.array(normal)

    
def New_Data_Generator_D(disorder):
    number_of_samples = 45
    randlist = (132 * np.random.random(number_of_samples)).astype(int)
    new_patients = list(disorder[i] for i in randlist)
    
    index = [ i for i in range(number_of_samples)]
    comb = list( combinations(index, 2))
    new_data = [[] for i in range(len(comb))]
    person = 0
    for x,y in comb:
        for chan in range(19):
            new_data[person].append((new_patients[x][chan] + new_patients[y][chan] )/2)
        disorder.append(new_data[person])
        person += 1
    
    return np.array(disorder)


In [0]:
normal_list, disorder_list = data_gen()


In [0]:
normal = New_Data_Generator_N(normal_list)
disorder = New_Data_Generator_D(disorder_list)

In [9]:
normal.shape

(1122, 19, 15360)

In [0]:
# the output of this function is the full data matrix and the target
#
def data_shaper(normal, disorder):
    # 0 is the disorder and the 1 is the normal
    subjects, channels, data = normal.shape[0], normal.shape[1], normal.shape[2] 
    target = np.vstack((np.ones((subjects,1)),np.zeros((subjects,1))))
    #data = np.zeros((2*subjects, data, channels))
    n=0
    d=0
    i=0
    
    
    data = np.vstack((normal, disorder))
    
    '''
    while (n != subjects or d != subjects) :
        rand = np.random.random()
        
        if rand < 0.5 and n !=subjects :  # for the normal patients
            data[i] = normal[n].T
            target[i] = 1
            n +=1
            i +=1
            #print('n={}'.format(n))
        if rand >= 0.5 and d !=subjects :  # for the disorder patients
            data[i] = disorder[d].T
            target[i] = 0
            d+=1
            i+=1
            #print('d={}'.format(d))
            
    '''
            
    index = shuffle(np.arange(2*subjects))
    
    data, target = shuffle(data, target)
    
            
    return data, to_categorical(target)
    

In [0]:
data , target = data_shaper(normal, disorder)

In [0]:
model = Sequential([
    Conv1D(filters=5, kernel_size=21, activation='relu', input_shape=(15360, 19)),
    MaxPooling1D(pool_size=7),
    Conv1D(filters=4, kernel_size=7, activation='relu', input_shape=(15360, 19)),
    MaxPooling1D(pool_size=7),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(10, activation='relu'),
    Dense(5, activation='relu'),
    Dense(2, activation='softmax'),
    
    
])

In [0]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [39]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_9 (Conv1D)            (None, 15340, 5)          2000      
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 2191, 5)           0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 2185, 4)           144       
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 312, 4)            0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 1248)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 100)               124900    
_________________________________________________________________
dense_16 (Dense)             (None, 10)               

In [40]:

his = model.fit(data, target, epochs=10, batch_size=1)

Epoch 1/10
1271/2244 [===============>..............] - ETA: 13s - loss: 7.8625 - acc: 0.5122

KeyboardInterrupt: ignored

In [16]:
x = np.arange(10)
shuffle(x)

array([5, 9, 8, 7, 1, 3, 0, 2, 6, 4])

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 15340, 5)          2000      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 2191, 5)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2185, 5)           180       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 312, 5)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1560)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                15610     
_________________________________________________________________
dense_2 (Dense)              (None, 5)                

In [0]:
ind= shuffle(np.arange(132*2))

In [0]:
index = np.arange(132*2)

In [0]:
ind = shuffle(index)

In [21]:
ind

array([203,  94,  16,  36,  30, 175,  78, 243, 171, 170,  81,  74, 125,
       259,  58,  47,  88, 253, 187,  35, 150, 121,  53,  86, 245, 219,
       148,  24, 228, 232, 222, 202,  20, 198,  97, 196, 122, 188, 117,
       166, 168, 213, 107, 263, 100,  89,  15,  84,  13, 191, 234, 258,
       208, 241,   1, 249,   0, 256,  70,  68, 261, 162,  22,  71,  76,
       114,  31, 120,  62,  55,  90,  42, 194,  73,  21, 138,  33, 177,
        67, 156,  45, 181,  79, 215, 237, 201,  52, 185,   6, 216, 227,
       212, 165, 210,  28,  99, 136, 155, 142,  32,  60,  77,  19, 209,
       199, 238,  41,  39,  54, 154,  11, 255, 247, 217, 111, 179,  49,
       130,   8,  12, 184,  29,   7, 123, 204, 182, 189,  72, 163, 251,
       239,  69,  64,   3, 118, 132, 207, 197, 159, 200, 115, 246, 174,
       149, 260, 153, 236, 134, 161, 127, 106,   2, 164, 223,  18, 129,
        26, 178, 224, 137, 101, 140, 144,  23,  40, 183,  75,  66,  95,
        87, 226, 104, 229, 186, 262, 103, 112,  48, 252, 257,  8

In [0]:
np.random.shuffle([1,2,3])

In [0]:
a = [1,2,3,4,5,6]
t = [2,3,1,2,3,4]
a,b = shuffle(a,t)

In [24]:
normal.shape

(1122, 19, 15360)

In [0]:
a = np.random.random((4,6,5))
b = np.random.random((8,6,5))

In [42]:
a.shape

(4, 6, 5)

In [43]:
b.shape

(8, 6, 5)

In [0]:
c = np.vstack((a,b))

In [49]:
c.shape

(12, 6, 5)

In [53]:
help(data_shaper)

Help on function data_shaper in module __main__:

data_shaper(normal, disorder)

